<a href="https://colab.research.google.com/github/angelohafner/linguagem-de-programacao-udesc/blob/main/Colab_Maclaurin_Series_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Série de Maclaurin no Colab 🎓🧮 (divertido e visual)

Bem-vindo(a)! Neste tutorial vamos **aprender e visualizar** como funcionam as **séries de Maclaurin** usando Python, com exemplos práticos e **widgets interativos**.  
Você vai:
- 📘 Entender o que é uma série de Maclaurin e por que ela aparece tanto na **física** e na **engenharia**;
- ✍️ Ver **exemplos passo a passo** usando `sympy` e `math`;
- 📊 Comparar a **função original** com as **aproximações por série**;
- 🎮 Brincar com **widgets** para escolher o valor de `x` e o **número de termos** `N`;
- 💡 Calcular somatórios clássicos: `exp`, `cos`, `sen`;
- 🚀 Resolver **desafios** finais (como `ln(1+x)` e `atan(x)`).

> Dica: Execute as células na ordem. Ajuste os sliders e observe como a aproximação melhora quando aumentamos o número de termos! 😄


## 1) O que é série de Maclaurin? 🤔

Uma **série de Maclaurin** é um caso particular da série de Taylor **em torno de 0**.  
Se \(f\) é infinitamente diferenciável em 0, então:
\[
f(x) = \sum_{n=0}^{\infty} \frac{f^{(n)}(0)}{n!}\, x^n
\]

**Onde aparece?**  
- Física: aproximações de soluções de EDOs, oscilações, óptica, mecânica quântica.  
- Engenharia: análise de sinais, controle (resposta de sistemas), eletrônica (aprox. exponenciais), numérico (lin. e não linearidades).

**Ideia-chave:** usar **poucos termos** perto de \(x=0\) para aproximar funções difíceis de calcular diretamente.


In [1]:
# Setup (Colab-ready)
# Comments are in English (ASCII only).
import math
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from ipywidgets import interact, FloatSlider, IntSlider, fixed

sp.init_printing()  # pretty printing for sympy

print("Ready! Sympy, numpy, matplotlib, ipywidgets imported.")


Ready! Sympy, numpy, matplotlib, ipywidgets imported.


## 2) Demonstração simbólica com Sympy ✍️

Vamos pedir à `sympy` as primeiras expansões de Maclaurin para `exp(x)`, `cos(x)` e `sin(x)`.


In [2]:
# Symbolic series using sympy
x = sp.Symbol('x')

exp_series = sp.series(sp.exp(x), x, 0, 8)   # up to x^7
cos_series = sp.series(sp.cos(x), x, 0, 10)  # up to x^9
sin_series = sp.series(sp.sin(x), x, 0, 9)   # up to x^8

print("exp(x) Maclaurin up to x^7:")
display(exp_series)
print("cos(x) Maclaurin up to x^9:")
display(cos_series)
print("sin(x) Maclaurin up to x^8:")
display(sin_series)


exp(x) Maclaurin up to x^7:


         2    3    4    5     6      7         
        x    x    x    x     x      x      ⎛ 8⎞
1 + x + ── + ── + ── + ─── + ─── + ──── + O⎝x ⎠
        2    6    24   120   720   5040        

cos(x) Maclaurin up to x^9:


     2    4    6      8           
    x    x    x      x       ⎛ 10⎞
1 - ── + ── - ─── + ───── + O⎝x  ⎠
    2    24   720   40320         

sin(x) Maclaurin up to x^8:


     3    5      7         
    x    x      x      ⎛ 9⎞
x - ── + ─── - ──── + O⎝x ⎠
    6    120   5040        

## 3) Construindo as somas parciais (passo a passo) 🧱

Abaixo, definimos funções para calcular as **somas parciais** (com `math`) e comparar com o valor exato (`math.exp`, `math.cos`, `math.sin`).  
Também medimos o **erro absoluto** no ponto escolhido.


In [8]:
# Partial sums and utilities (math-based)
# All comments in English ASCII only.

def maclaurin_exp_partial(xval, N):
    # Compute partial sum S_N for exp(x) = sum_{n=0..N} x^n/n!
    total = 0.0
    n = 0
    while n <= N:
        term = (xval ** n) / math.factorial(n)
        total = total + term
        n = n + 1
    return total

def maclaurin_cos_partial(xval, N):
    # Compute partial sum S_N for cos(x) = sum_{n=0..N} (-1)^n x^(2n)/(2n)!
    total = 0.0
    n = 0
    while n <= N:
        term = ((-1.0) ** n) * (xval ** (2 * n)) / math.factorial(2 * n)
        total = total + term
        n = n + 1
    return total

def maclaurin_sin_partial(xval, N):
    # Compute partial sum S_N for sin(x) = sum_{n=0..N} (-1)^n x^(2n+1)/(2n+1)!
    total = 0.0
    n = 0
    while n <= N:
        term = ((-1.0) ** n) * (xval ** (2 * n + 1)) / math.factorial(2 * n + 1)
        total = total + term
        n = n + 1
    return total

def plot_function_vs_partial(func_name, x_select, N, x_min, x_max):
    # Plot the true function vs Maclaurin partial sum on a single chart.
    xs = np.linspace(x_min, x_max, 800)

    if func_name == 'exp':
        y_true = np.exp(xs)
        y_approx = np.zeros_like(xs)
        i = 0
        while i < xs.size:
            y_approx[i] = maclaurin_exp_partial(xs[i], N)
            i = i + 1
        y_true_sel = math.exp(x_select)
        y_approx_sel = maclaurin_exp_partial(x_select, N)
        title = "exp(x)"
    elif func_name == 'cos':
        y_true = np.cos(xs)
        y_approx = np.zeros_like(xs)
        i = 0
        while i < xs.size:
            y_approx[i] = maclaurin_cos_partial(xs[i], N)
            i = i + 1
        y_true_sel = math.cos(x_select)
        y_approx_sel = maclaurin_cos_partial(x_select, N)
        title = "cos(x)"
    else:
        y_true = np.sin(xs)
        y_approx = np.zeros_like(xs)
        i = 0
        while i < xs.size:
            y_approx[i] = maclaurin_sin_partial(xs[i], N)
            i = i + 1
        y_true_sel = math.sin(x_select)
        y_approx_sel = maclaurin_sin_partial(x_select, N)
        title = "sin(x)"

    err = abs(y_true_sel - y_approx_sel)

    plt.figure()
    plt.plot(xs, y_true, label="true function")
    plt.plot(xs, y_approx, linestyle="--", label=f"Maclaurin partial (N={N})")
    plt.axvline(x_select, linestyle=":", linewidth=1)
    plt.axhline(0, linewidth=1)
    plt.axvline(0, linewidth=1)
    plt.grid(True)
    plt.title(f"{title} vs Maclaurin (error at x={x_select:.2f} is {err:.2e})")
    plt.legend()
    plt.show()


## 4) Interativo: `exp(x)` 🚀

A série de Maclaurin de `exp(x)` é:
\[
\sum_{n=0}^{\infty} \frac{x^n}{n!}
\]

Mova os sliders para escolher `x` e o número de termos `N`. Observe o erro indicado no título.


In [4]:
# Interactive exp
@interact(
    x_select=FloatSlider(value=1.0, min=-5.0, max=5.0, step=0.1, description="x"),
    N=IntSlider(value=5, min=0, max=30, step=1, description="N")
)
def interactive_exp(x_select, N):
    plot_function_vs_partial('exp', x_select, N, -5.0, 5.0)


interactive(children=(FloatSlider(value=1.0, description='x', max=5.0, min=-5.0), IntSlider(value=5, descripti…

## 5) Interativo: `cos(x)` 🌊 (parte par)

A série de Maclaurin de `cos(x)` é:
\[
\sum_{n=0}^{\infty} \frac{(-1)^n x^{2n}}{(2n)!}
\]

Explore como as **potências pares** aparecem e como o sinal alterna.


In [5]:
# Interactive cos
@interact(
    x_select=FloatSlider(value=1.0, min=-5.0, max=5.0, step=0.1, description="x"),
    N=IntSlider(value=5, min=0, max=30, step=1, description="N")
)
def interactive_cos(x_select, N):
    plot_function_vs_partial('cos', x_select, N, -5.0, 5.0)


interactive(children=(FloatSlider(value=1.0, description='x', max=5.0, min=-5.0), IntSlider(value=5, descripti…

## 6) Interativo: `sin(x)` 🌊 (parte ímpar)

A série de Maclaurin de `sin(x)` é:
\[
\sum_{n=0}^{\infty} \frac{(-1)^n x^{2n+1}}{(2n+1)!}
\]

Note como somente **potências ímpares** aparecem.


In [6]:
# Interactive sin
@interact(
    x_select=FloatSlider(value=1.0, min=-5.0, max=5.0, step=0.1, description="x"),
    N=IntSlider(value=5, min=0, max=30, step=1, description="N")
)
def interactive_sin(x_select, N):
    plot_function_vs_partial('sin', x_select, N, -5.0, 5.0)


interactive(children=(FloatSlider(value=1.0, description='x', max=5.0, min=-5.0), IntSlider(value=5, descripti…

## 7) Conferindo os somatórios clássicos 💡

Vamos calcular numericamente os somatórios **no ponto x escolhido** e comparar com `math.exp`, `math.cos`, `math.sin`.


In [7]:
# Numeric check at a single x
# User picks x and N_max; we show the partial sums and the true values.
@interact(
    xval=FloatSlider(value=1.0, min=-4.0, max=4.0, step=0.1, description="x"),
    Nmax=IntSlider(value=10, min=1, max=50, step=1, description="N_max")
)
def check_summations(xval, Nmax):
    s_exp = maclaurin_exp_partial(xval, Nmax)
    s_cos = maclaurin_cos_partial(xval, Nmax)
    s_sin = maclaurin_sin_partial(xval, Nmax)

    true_exp = math.exp(xval)
    true_cos = math.cos(xval)
    true_sin = math.sin(xval)

    err_exp = abs(true_exp - s_exp)
    err_cos = abs(true_cos - s_cos)
    err_sin = abs(true_sin - s_sin)

    print(f"exp series partial (N={Nmax}) at x={xval:.3f} = {s_exp:.12f}")
    print(f"math.exp(x)                        = {true_exp:.12f}")
    print(f"abs error                          = {err_exp:.3e}\n")

    print(f"cos series partial (N={Nmax}) at x={xval:.3f} = {s_cos:.12f}")
    print(f"math.cos(x)                        = {true_cos:.12f}")
    print(f"abs error                          = {err_cos:.3e}\n")

    print(f"sin series partial (N={Nmax}) at x={xval:.3f} = {s_sin:.12f}")
    print(f"math.sin(x)                        = {true_sin:.12f}")
    print(f"abs error                          = {err_sin:.3e}")


interactive(children=(FloatSlider(value=1.0, description='x', max=4.0, min=-4.0), IntSlider(value=10, descript…

## 8) Nota aplicada ⚙️

- **Controle/Engenharia elétrica:** exponenciais aparecem na resposta transitória de circuitos `RC/RL` e em funções de transferência.  
- **Vibrações/ondas:** `sen` e `cos` descrevem soluções harmônicas; séries ajudam a aproximar soluções quando só temos termos próximos de \(x=0\).  
- **Computação científica:** bibliotecas numéricas implementam `exp`, `sin`, `cos` com técnicas relacionadas a séries e polinômios aproximantes.


# 9) Desafios finais 🚀

Tente implementar as séries e gráficos **no mesmo formato** (com sliders `x` e `N`) para:

1. **\(\ln(1+x)\)** (para \(|x|<1\)):
\[
\ln(1+x) = \sum_{n=1}^{\infty} \frac{(-1)^{n+1}}{n} x^n
\]
   - Dica: verifique o **raio de convergência**.
2. **\(\arctan(x)\)** (para \(|x|\le 1\), converge lentamente em \(|x|=1\)):
\[
\arctan(x) = \sum_{n=0}^{\infty} \frac{(-1)^n}{2n+1} x^{2n+1}
\]
3. **Geometria (revisão):** \(\frac{1}{1-x} = \sum_{n=0}^{\infty} x^n\) para \(|x|<1\). Use isso para criar aproximações racionais simples.

> Extra: Plote o **erro absoluto** em função de `N` para um `x` fixo e discuta a rapidez de convergência. 🎯
